In [1]:
import tensorflow as tf
import time
from object_detection.utils import config_util
from object_detection.builders import model_builder
import cv2

2024-05-19 14:29:40.649914: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-19 14:29:40.650128: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-19 14:29:40.819664: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-19 14:29:41.207030: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-19 14:29:46.080130: W tensorflow/compiler/tf2

In [2]:
# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

2024-05-19 14:30:00.162561: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-19 14:30:00.663053: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-19 14:30:00.663245: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
#model_name = "my_ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8"
#model_name = "my_ssd_resnet50_v1_fpn"
model_name = "my_ssd_resnet50_v1_fpn_exported"
#model_name = "my_ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8"
#model_name = "my_ssd_resnet152_v1_fpn_640x640_coco17_tpu-8"
#model_name = "my_ssd_resnet101_v1_fpn_640x640_coco17_tpu-8"

images_path = f"TensorFlow/workspace/training_demo/images"
model_path = f"TensorFlow/workspace/training_demo/models/{model_name}"
exported_model_path = f"TensorFlow/workspace/training_demo/exported-models/"

ckpt_dict = {"my_ssd_resnet50_v1_fpn":'/ckpt-31',
                    "my_ssd_resnet101_v1_fpn_640x640_coco17_tpu-8":"/ckpt-28",
                    "my_ssd_resnet152_v1_fpn_640x640_coco17_tpu-8":"/ckpt-26",
                    "my_ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8":"/ckpt-26",
                    "my_ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8": "/ckpt-51"}

#### Load Data

In [4]:
def find_images_names(path):
    images_names = []
    
    for image_name in os.listdir(path):
        if image_name.endswith(('.bmp', '.jpg', '.png', '.jpeg')):
            images_names.append(image_name)
    
    return images_names

train_images_name = find_images_names(images_path + "/train")
train_images = []
for image_name in train_images_name:
    train_images(np.array(cv2.imread(images_path + "/train/" + image_name)))

test_images_name = find_images_names(images_path + "/test")
test_images = []
for image_name in test_images_name:
    test_images(np.array(cv2.imread(images_path + "/test/" + image_name)))

NameError: name 'os' is not defined

#### Teacher Model

In [ ]:
if model_name[-9:] == "_exported":
    print("[Loading TF2 Saved Model]")
    print(f'Loading model - {model_name}...')
    start_time = time.time()

    teacher_model = tf.saved_model.load(exported_model_path + f"{model_name}/saved_model")

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f'The model has been loaded - {elapsed_time:.2f}s\n')
else:
    tf.get_logger().setLevel('ERROR')           # Suppress TensorFlow logging (2)
    print("[Loading TF2 Checkpoint]")
    print(f'\nLoading model - {model_name}...')
    start_time = time.time()

    # Load pipeline config and build a detection model
    configs = config_util.get_configs_from_pipeline_file(model_path + "/pipeline.config")
    model_config = configs['model']

    teacher_model = model_builder.build(model_config=model_config, is_training=False)

    # Restore checkpoint
    ckpt = tf.compat.v2.train.Checkpoint(model=teacher_model)
    ckpt.restore(model_path + ckpt_dict[model_name]).expect_partial()

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f'The model has been loaded - {elapsed_time:.2f}s\n')

[Loading TF2 Saved Model]
Loading model - my_ssd_resnet50_v1_fpn_exported...
The model has been loaded - 30.75s



#### Student Model

In [ ]:
student_model = tf.keras.applications.MobileNet(
        input_shape=(640, 640, 3),
        include_top=True,
        weights=None,
        classes=1
)